# Functions.py

In [1]:
def extract_data(xml_root_element):

    """ Function to extract the data from the first tag of the XML file string, contained in the main root (Response). It returns a dictionary 
    having the tags (attribute .tag) of the child elements (for example "donor age" in <donor--age>25 yrs</donor--age>) as keys and 
    the text content (attribute .text) as value ("25 yrs" in <donor--age>25 yrs</donor--age>). 
    -> This function can be executed inside the create_dataframe() function of function.py"""
    
    data = {}
    for child in xml_root_element:
        data[child.tag] = child.text
    return data     

In [2]:
def create_dataframe(first_tag:str):
    """ The function creates a dataframe from XLM files from which the data were extracted with extract_data() function.
    As argument it takes the first tag (string) within the root that incorporates all the elements associated"""
    df_rows = []
    for x in root.findall(first_tag):
        df_rows.append(fun.extract_data(x))

    df = pd.DataFrame(df_rows)

    return df

To exctract data and obtain a Pandas DataFrame from an API returning XML files:

1) import pandas, requests and xml.etree.ElementTree libraries
2) import functions as fun (import the functions of functions.py, use "fun" as alias)
3) define url
4) define criteria to extract specific data (the criteria are specified in the Allen Brain website as RMA queries and are strings)
5) create a variable "response" that stores the get request taking as argument the url and as params the criteria (params = {"criteria":criteria})
6) check the status_code of "response" (has to be 200, so "approved")
7) create the variable "data" that stored the XML file ("response") as text (using the attribute .text)
8) Find the main root with ElementTree library method et.fromstring()
9) Look at the first lines of XML document to find the first tag within the root that incorporates all elements (usually the tag in the second line) -> print(data)[:10]
10) Execute the function create_dataframe() taking as argument the first tag found at point 9 as string (".//first_tag")
11) Save the dataframe as "raw_data.csv" -> df.to_csv("raw_data.csv")
12) create a copy of the raw dataframe and work on it!

### Cleaning and formatting data in different columns

In [4]:
def rename_ef(df):
    "Function that renames the column starting with ef-- (=electrophysiology), by removing ef--"""
    df.columns = [x.replace("ef--", "") if x.startswith("ef--") else x for x in df.columns]

    return df

In [9]:
def n_age(df):
    """ Function to format age. Age data are strings in the format of "n yrs" (string), 
    thus the function removes "yrs" and converts the numbers to integers. 
    The function handles NaN values and leaves them as they are, if present"""
    
    df["donor--age"] = df["donor--age"].str.replace(" yrs", "")
    #df["donor--age"] = [int(x) if pd.notnull(x) else np.nan for x in df["donor--age"]]
    df["donor--age"] = df["donor--age"].apply(lambda x: int(x) if pd.notnull(x) else np.nan)

    return df

In [11]:
def age_category(age):
    """Functions that creates age categories. 
    It returns different categories of age, so can be applied to the column "donor--age" 
    and to create a new column where each donor has an assignedage category"""

    if isinstance(age, int):
        if 18 <= age <= 24:
            return "18-24 years"
        elif 25 <= age <= 30:
            return "25-30 years"
        elif 31 <= age <= 40:
            return "31-40 years"
        elif 41 <= age <= 50 :
            return "41-50 years"
        elif 51 <= age <= 65:
            return "51-65 years"
        elif 66 <= age <= 83:
            return "66-83 years"
        else:
            return np.nan
    else:
        return np.nan

In [6]:
def format_firing(df):
    """ Function that converts the firing rates object in floats with two decimals. 
    It handles NaN values and leaves them as they are, if present"""
    
    df["avg-firing-rate"] = df["avg-firing-rate"].apply(lambda x: round(float(x), 2) if pd.notnull(x) else np.nan)

    return df

In [7]:
def format_curve_slope(df):
    """ Function that converts values of f-i curve slope from strings to floats with 3 decimals.
    It handles NaN values and leaves them as they are, if present"""
    
    df["f-i-curve-slope"] = df["f-i-curve-slope"].apply(lambda x: round(float(x), 3) if pd.notnull(x) else np.nan)

    return df